In [1]:
import pandas as pd
import numpy as np

In [2]:
path = 'E:/my_courses/data mining/project/data_genre/'
path = "C:/Users/User/Downloads/test2/"

In [3]:
rating = pd.read_csv(path + 'ratings.csv')
to_read = pd.read_csv(path + 'to_read.csv')
book = pd.read_csv(path + 'book_no_null_language_merging.csv')

# 4️⃣ Setup & Modeling & Evaluation Phase
<hr style="border:2px solid gray"> </hr>

## 📑 AssociationRules

In [104]:
to_read.head()

,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


In [105]:
rating.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [106]:
to_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912705 entries, 0 to 912704
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user_id  912705 non-null  int64
 1   book_id  912705 non-null  int64
dtypes: int64(2)
memory usage: 13.9 MB


In [107]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB


In [108]:
to_read.isna().sum()

user_id    0
book_id    0
dtype: int64

In [109]:
rating.isna().sum()

user_id    0
book_id    0
rating     0
dtype: int64

### ✅ Creating one hot from to_read & ratings datasets

<div style="direction:rtl", align="justify">
<h3>
<span style='font-family:"B Nazanin"'>
در این بخش به عنوان گامی از فاز مدلسازی قصد داریم ارتباط میان کتاب های گوناگونی که توسط کاربران خوانده شده است را کشف نماییم. بدین منظور ابتدا جدول one_hot ایجاد میکنیم که ستون های نظیر آن id مربوط به ده هزار کتاب موجود در دیتاست book باشد و سپس به ازای هر کاربر یک رکورد در جدول ایجاد کرده و برای آن رکورد مقدار ستون نظیر کتابهایی که کاربر مربوطه آن ها را خوانده و یا علاقمند به خواندن آن ها بوده 1 و در مقابل سایر کتاب ها 0 قرار می دهیم.

In [110]:
read_1_hot = pd.DataFrame(0,columns=[i for i in range(1,10001)],index=[i for i in range(1,53425)])

In [111]:
for _uid_ in to_read['user_id'].unique():
    x = to_read[to_read['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        read_1_hot.loc[_uid_, _bid_] = 1

In [112]:
for _uid_ in rating['user_id'].unique():
    x = rating[rating['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        read_1_hot.loc[_uid_, _bid_] = 1

In [113]:
read_1_hot

,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,1,1,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
53421,1,1,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
53422,1,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
53423,1,1,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
read_1_hot.to_csv(path + 'to_read_1_hot.csv')

In [115]:
import gc
gc.collect()

7015

### ✅ Which books are related

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
از جدول one_hot بدست آمده در مرحله قبل استفاده کرده و با استفاده از الگوریتم fpgrowth و با minsup = 0.1 آیتم ست های متناوب موجود در این جدول را استخراج می کنیم. با توجه به اینکه دیتاست ما حاوی بیش از 53,000 کاربر است مقدار minsup = 0.1 عنوان می کند که آیتم ست استخراج شده می بایست حداقل در 10 درصد از کل کاربران صادق باشد یعنی بیش از حدود 5300 نفر همه کتاب های موجود در آیتم ست های استخراج شده را مطالعه کرده باشند. با توجه به آزمون و خطا های صورت گرفته اگر حدود 10 درصد از افراد دو کتاب x و y را باهم مطالعه کرده باشند این قاعده قابل بررسی و حائز اهمیت است.  

In [2]:
import pandas as pd
import numpy as np

read_1_hot = pd.read_csv(path+'to_read_1_hot.csv')
read_1_hot.drop(['Unnamed: 0'], axis=1, inplace=True)

In [118]:
read_1_hot.head()

,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
import gc
gc.collect()

3716904

In [119]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [120]:
frq_items_fpgrowth = fpgrowth(read_1_hot, min_support = 0.1, use_colnames = True)

frq_items_fpgrowth.sort_values(by=['support'], inplace=True)

In [122]:
frq_items_fpgrowth.shape

(2071, 2)

In [123]:
frq_items_fpgrowth

,support,itemsets
1735,0.100011,"(2, 18, 27, 26)"
1004,0.100011,"(2, 23, 18, 21, 20, 25)"
1227,0.100011,"(2, 23, 21, 27, 7)"
1586,0.100030,"(2, 23, 21, 20, 27, 25)"
696,0.100030,"(2, 20, 25, 24, 1, 17)"
...,...,...
120,0.322271,(3)
58,0.334924,(5)
0,0.384864,(4)
57,0.416348,(2)


In [124]:
book_rules = association_rules(frq_items_fpgrowth, metric ="lift", min_threshold = 1)

<div style="direction:rtl", align="justify"><h3>
     <span style='font-family:"B Nazanin"'>   
در مجموع از روی frequent_itemset های موجود که حاوی minsup = 0.1 هستند تعداد 44452 قاعده استخراج شده است و تنها قواعدی که lift بالای 1 دارند مدنظر ما می باشد.

In [125]:
book_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(2, 27, 18)",(26),0.215690,0.253051,0.100011,0.463681,1.832363,0.045431,1.392733
1,"(2, 18, 26)",(27),0.110344,0.290450,0.100011,0.906361,3.120542,0.067962,7.577531
2,"(2, 27, 26)",(18),0.104541,0.304676,0.100011,0.956670,3.139959,0.068160,16.047048
3,"(27, 18, 26)",(2),0.116783,0.416348,0.100011,0.856387,2.056900,0.051389,4.064065
4,"(2, 18)","(27, 26)",0.244272,0.126123,0.100011,0.409425,3.246236,0.069203,1.479706
...,...,...,...,...,...,...,...,...,...
44447,(23),(18),0.300558,0.304676,0.276655,0.920471,3.021148,0.185082,8.743008
44448,(24),(18),0.298518,0.304676,0.276767,0.927138,3.043032,0.185816,9.543055
44449,(18),(24),0.304676,0.298518,0.276767,0.908398,3.043032,0.185816,7.657968
44450,(17),(20),0.320792,0.311339,0.287062,0.894854,2.874205,0.187187,6.549536


In [126]:
book_rules.to_csv(path + 'book_rules.csv')

### ✅ Creating to_read_authors dataset using to_read & ratings datasets

<div style="direction:rtl", align="justify"><h3>
<span style='font-family:"B Nazanin"'>
در این بخش قصد داریم ارتباط میان نویسندگانی که کتاب های آن ها معمولا باهم توسط یک فرد مطالعه می شود را کشف کنیم. بدین منظور یک جدول one_hot تشکیل می دهیم که ستون های آن نام تمام نویسندگانی است که حداقل یک کتاب از آن ها در دیتاست book موجود می باشد و به ازای هر کاربر یک رکورد در جدول ایجاد کرده و مقدار نظیر نویسندگانی که از آن ها کتابی را مطالعه کرده است 1 و در مقابل نام سایر نویسندگان به ازای آن رکورد 0 قرار می دهیم.

In [127]:
import re
unique_authors = set()
for i in book['authors'].unique():
    res = re.split(',', i)
    for j in res:
        unique_authors.add(j.lstrip())

In [128]:
len(unique_authors)

5841

In [129]:
authors_1_hot = pd.DataFrame(0,columns=[i for i in unique_authors],index=[i for i in range(0,53425)])

In [130]:
for _uid_ in to_read['user_id'].unique():
    x = to_read[to_read['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        authors = re.split(',', book.loc[_bid_ -1, 'authors'])
        for _au_ in authors:
            authors_1_hot.loc[_uid_, _au_.lstrip()] = 1

In [131]:
for _uid_ in rating['user_id'].unique():
    x = rating[rating['user_id'] == _uid_]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        authors = re.split(',', book.loc[_bid_ -1, 'authors'])
        for _au_ in authors:
            authors_1_hot.loc[_uid_, _au_.lstrip()] = 1

In [132]:
authors_1_hot

,Rachel Hauck,John Fowles,Amy Schumer,Jerzy Kosiński,Sonya Sones,Peggy Fortnum,Crystal S. Chan,Bill Moyers,Harvey Karp,Steven Naifeh,...,Mikhail Lermontov,Mourid Barghouti,Sarah Palin,Alex Tresniowski,Sandra Dallas,Eithne Wilkins,Lacey Weatherford,Peter Lerangis,Frank Bettger,محمد اسماعیل‌زاده
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
authors_1_hot.to_csv(path + 'authors_1_hot.csv')

In [ ]:
del authors_1_hot

### ✅ Which authors are related

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
از جدول one_hot بدست آمده در مرحله قبل استفاده کرده و با استفاده از الگوریتم fpgrowth و با minsup = 0.25 آیتم ست های متناوب موجود در این جدول را استخراج می کنیم. با توجه به اینکه دیتاست ما حاوی بیش از 53,000 کاربر است مقدار minsup = 0.25 عنوان می کند که آیتم ست استخراج شده می بایست حداقل در 25 درصد از کل کاربران صادق باشد یعنی بیش از حدود 13250 نفر حداقل یک کتاب از هر نویسنده موجود در آیتم ست های استخراج شده را مطالعه کرده باشند. با توجه به آزمون و خطا های صورت گرفته اگر حدود 25 درصد از افراد از دو نویسنده x و y به طور همزمان کتاب مطالعه کرده باشند این قاعده  قابل بررسی و حائز اهمیت است. 

In [135]:
import pandas as pd
import numpy as np

authors_1_hot = pd.read_csv(path + 'authors_1_hot.csv')
authors_1_hot.drop(['Unnamed: 0'], axis=1, inplace=True)

In [136]:
authors_1_hot.head()

,Rachel Hauck,John Fowles,Amy Schumer,Jerzy Kosiński,Sonya Sones,Peggy Fortnum,Crystal S. Chan,Bill Moyers,Harvey Karp,Steven Naifeh,...,Mikhail Lermontov,Mourid Barghouti,Sarah Palin,Alex Tresniowski,Sandra Dallas,Eithne Wilkins,Lacey Weatherford,Peter Lerangis,Frank Bettger,محمد اسماعیل‌زاده
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [137]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [138]:
frq_items_fpgrowth = fpgrowth(authors_1_hot, min_support = 0.25, use_colnames = True)

frq_items_fpgrowth.sort_values(by=['support'], inplace=True)

In [139]:
frq_items_fpgrowth.shape

(72, 2)

In [140]:
frq_items_fpgrowth

,support,itemsets
28,0.250108,(Garth Williams)
63,0.252410,"(Guy Cardwell, Mark Twain, John Seelye)"
62,0.252410,"(Guy Cardwell, Mark Twain)"
61,0.252410,"(Guy Cardwell, John Seelye)"
27,0.252410,(Guy Cardwell)
...,...,...
29,0.435470,(J.R.R. Tolkien)
51,0.518390,"(Mary GrandPré, J.K. Rowling)"
17,0.518390,(Mary GrandPré)
32,0.520056,(Suzanne Collins)


In [141]:
author_rules = association_rules(frq_items_fpgrowth, metric ="lift", min_threshold = 1)

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
در مجموع از روی frequent_itemset های موجود که حاوی minsup = 0.25 هستند تعداد 106 قاعده استخراج شده است و تنها قواعدی که lift بالای 1 دارند مدنظر ما می باشد.

In [142]:
author_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Guy Cardwell, Mark Twain)",(John Seelye),0.252410,0.259785,0.252410,1.000000,3.849341,0.186838,inf
1,"(Guy Cardwell, John Seelye)",(Mark Twain),0.252410,0.276743,0.252410,1.000000,3.613460,0.182557,inf
2,"(Mark Twain, John Seelye)",(Guy Cardwell),0.253046,0.252410,0.252410,0.997485,3.951846,0.188539,297.255008
3,(Guy Cardwell),"(Mark Twain, John Seelye)",0.252410,0.253046,0.252410,1.000000,3.951846,0.188539,inf
4,(Mark Twain),"(Guy Cardwell, John Seelye)",0.276743,0.252410,0.252410,0.912073,3.613460,0.182557,8.502399
...,...,...,...,...,...,...,...,...,...
101,(J.K. Rowling),"(Mary GrandPré, Suzanne Collins)",0.613552,0.335929,0.335929,0.547515,1.629854,0.129819,1.467610
102,(Suzanne Collins),(J.K. Rowling),0.520056,0.613552,0.376172,0.723330,1.178923,0.057091,1.396784
103,(J.K. Rowling),(Suzanne Collins),0.613552,0.520056,0.376172,0.613106,1.178923,0.057091,1.240505
104,(Mary GrandPré),(J.K. Rowling),0.518390,0.613552,0.518390,1.000000,1.629854,0.200331,inf


In [143]:
author_rules.to_csv(path+'author_rules.csv')

In [ ]:
del author_rules

## 📑 Classification

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
    در این قسمت به ساخت مدل میپردازیم . داده های بالانس شده در قسمت قبل را به صورت کامل به ورودی مدل میدهیم . 

In [9]:
balanced_final = pd.read_csv(path + 'balanced_classifier_input.csv')

In [10]:
authors = pd.read_csv(path + 'author_class.csv')

In [11]:
authors['book_id']=[i for i in range(1,10001)]

In [12]:
print(list(balanced_final.columns))

['user_id', 'book_id', 'rating', '1', '2', '3', '4', '5', '6', 'fantasy, paranormal', 'non-fiction', 'mystery, thriller, crime', 'fiction', 'young-adult', 'children', 'poetry', 'comics, graphic', 'romance', 'history, historical fiction, biography', 'ara', 'en', 'fre', 'ger', 'ind', 'jpn', 'other', 'per', 'pol', 'por', 'spa']


In [13]:
balanced_final.shape

(7148939, 30)

### ✅ Add authors class to final dataset

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
        در این قسمت دیتاست نهایی را با دیتاست نویسنده ها که ساخته بودیم جوین میزنیم که کلاس های نویسنده ی هر کتاب در دیتاست نهایی مشخص گردد.
        به علت اینکه این دیتاست به صورت one-hot است ، از pca زدن روی ستون های آن خودداری کردیم چون تاثیری نخواهد داشت .
  

In [14]:
balanced_result = pd.merge(balanced_final, authors,on=['book_id'])

In [15]:
balanced_result.shape

(7148939, 34)

In [16]:
balanced_result.head()

,user_id,book_id,rating,1,2,3,4,5,6,"fantasy, paranormal",...,jpn,other,per,pol,por,spa,class_0,class_1,class_2,class_others
0,1,258,5,0.10183,0.039319,-0.046848,0.028769,0.017864,0.000186,1,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0
1,11,258,3,0.10183,0.039319,-0.046848,0.028769,0.017864,0.000186,1,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0
2,143,258,4,0.10183,0.039319,-0.046848,0.028769,0.017864,0.000186,1,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0
3,242,258,5,0.10183,0.039319,-0.046848,0.028769,0.017864,0.000186,1,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0
4,325,258,4,0.10183,0.039319,-0.046848,0.028769,0.017864,0.000186,1,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0


<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
همه ی ستون ها به جز ستون rating که حکم متغیر هدف دارد را داخل متغیر x_train میریزیم .
        متغیر هدف که همان rating است داخل y_train میریزیم.
        متغیر هدف در واقع به ازای هر کتاب و user یک امتیاز بین 1 تا 5 به کتاب اختصاص داده است که این امتیاز را user به کتاب داده است . 
        حال هدف ما این است که با دادن user و ویژگی کتاب به مدل ، امتیازی که احتمال دارد این user به کتاب مورد نظر بدهد را پیش بینی کنیم 
        پس این مسئله از نوع classification است .

## ✏️ Test model on train and test data

## ⛔ Not used in the rest of code because this is the test

In [13]:

x_train = balanced_result.drop(columns=['rating']).copy()
y_train = balanced_result[['rating']].copy()

In [14]:
x_train['u_2'] = x_train.user_id.copy()
x_train['u_3'] = x_train.user_id.copy()
x_train['u_4'] = x_train.user_id.copy()
x_train['u_5'] = x_train.user_id.copy()
x_train['u_6'] = x_train.user_id.copy()
x_train['u_7'] = x_train.user_id.copy()
x_train['u_8'] = x_train.user_id.copy()

### ✅ Separating test and train data

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = 5)

### ✅ Check distribution of target variable in test and train data

In [16]:
print(Y_train['rating'].value_counts() / Y_train['rating'].value_counts().sum() * 100)

4    29.918986
5    27.747545
3    19.174105
2    11.999071
1    11.160293
Name: rating, dtype: float64


In [17]:
print(Y_test['rating'].value_counts() / Y_test['rating'].value_counts().sum() * 100)

4    29.927933
5    27.708234
3    19.186061
2    12.003738
1    11.174034
Name: rating, dtype: float64


<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
توزیع متغیر هدف در دو داده تست و یادگیری حدودا یکسان می باشد.

In [19]:
del balanced_result
del balanced_final

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense

In [26]:
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.utils import to_categorical

Y_train_1 = to_categorical(Y_train)[:,1:].copy()

### ✅ Building model (using keras library)

In [28]:
# define the keras model
model_test = Sequential()
model_test.add(Dense(20, input_dim=len(X_train.columns), activation='relu',kernel_initializer='random_normal',bias_initializer='zeros'))
model_test.add(Dense(30, activation='relu'))
model_test.add(Dense(20, activation='tanh'))
model_test.add(Dense(30, activation='relu'))
model_test.add(Dense(5, activation='softmax'))
# compile the keras model
# myoptimizer = optimizers.Adam(learning_rate=0.001)
opt = SGD(learning_rate=0.001, momentum=0.9)
model_test.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit the keras model on the dataset
history_test = model_test.fit(X_train, Y_train_1, epochs=4, batch_size=100000,shuffle=True)

Epoch 1/4
58/58 [==============================] - 25s 95ms/step - loss: 1.5827 - accuracy: 0.2686
Epoch 2/4
58/58 [==============================] - 5s 88ms/step - loss: 1.5343 - accuracy: 0.2972
Epoch 3/4
58/58 [==============================] - 5s 85ms/step - loss: 1.5337 - accuracy: 0.2974
Epoch 4/4
58/58 [==============================] - 5s 90ms/step - loss: 1.5335 - accuracy: 0.2976


In [30]:
y_pred_pro_test = model_test.predict(X_test)
y_pred_class_test = [np.ceil(np.median(np.where((y_pred_pro_test[i] >= y_pred_pro_test[i].mean()))[0])) 
                     for i in range(len(y_pred_pro_test))]

In [32]:
y_pred_orig_test = pd.DataFrame()
y_pred_orig_test['origin'] = np.ravel(Y_test)
y_pred_orig_test['predict'] = y_pred_class_test

### ✅ We use a trick to find the needed accuracy

In [216]:
y_pred_orig_test['result'] = np.zeros((len(y_pred_orig_test),1))
y_pred_orig_test['result'][abs(y_pred_orig_test['predict'] - y_pred_orig_test['origin']) <= 1] = 1

100 * y_pred_orig_test['result'].sum() / len(y_pred_orig_test)

76.80551242561835


<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
        اضافه کردن ستون های کپی از ستون ایدی یوزر به دیتاست باعث شد دقت کمی افزایش یافت در واقع از 75.7 به 76.8 افزایش یافت زیرا بر اساس فاز EDA متوجه شدیم هیچ متغیری به اندازه user بر خروجی موثر نیست.

## ⛔ Above code is not used in the rest of code because this is the test

In [17]:
from sklearn.model_selection import train_test_split

x_train = balanced_result.drop(columns=['rating']).copy()
y_train = balanced_result[['rating']].copy()

In [18]:
print(y_train['rating'].value_counts() / y_train['rating'].value_counts().sum() * 100)

4    29.920776
5    27.739683
3    19.176496
2    12.000004
1    11.163041
Name: rating, dtype: float64


In [19]:
del balanced_result
del balanced_final

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
    برای مدلسازی از الگوریتم شبکه عصبی استفاده میکنیم تا بتوان داده های زیاد را روی آن تست کرد .
    از کتابخانه keras که بسیار در این زمینه قوی است استفاده میکنیم.
    تعداد لایه ها و نورون های مختلف را تست کردیم و بهترین دقت را در مدلی بدست آوردیم که شامل 4 لایه ی پنهان و تعداد نورون به ترتیب 20و30و20و30 است.
    از اکتیویشن فانکشن های مختلف استفاده کردیم و در مدل از relu , tanh , sigmoid بهره بردیم و از ترکیب این سه تابع توانستیم دقت خوبی کسب کنیم .
        مدل را با SGD optimizer و معیار accuracy کامپایل کردیم .
        در شبکه عصبی معیاری به نام batch size برای داده های بزرگ استفاده میشود که همزمان همه ی داده ها را پردازش نمیکند. 
        برای اینکه تعداد داده ها حدود 7 میلیون شده است ، هر 100,000 یکبار مدل را اجرا میکند درواقع batch size
        برابر با 100,000 است.

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense

In [22]:
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.utils import to_categorical

y_train_1 = to_categorical(y_train)[:,1:].copy()

In [23]:
import gc
gc.collect()

30

### ✅ We add some copies of user_id column 
### in order to increase effect of mentioned column in learning model

In [24]:
x_train['u_2'] = x_train.user_id.copy()
x_train['u_3'] = x_train.user_id.copy()
x_train['u_4'] = x_train.user_id.copy()
x_train['u_5'] = x_train.user_id.copy()
x_train['u_6'] = x_train.user_id.copy()
x_train['u_7'] = x_train.user_id.copy()
x_train['u_8'] = x_train.user_id.copy()

In [25]:
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=len(x_train.columns), activation='relu',kernel_initializer='random_normal',bias_initializer='zeros'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(30, activation='relu'))
model.add(Dense(5, activation='softmax'))
# compile the keras model
# myoptimizer = optimizers.Adam(learning_rate=0.001)
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit the keras model on the dataset
history = model.fit(x_train, y_train_1, epochs=4, batch_size=100000,shuffle=True)

Epoch 1/4
72/72 [==============================] - 20s 89ms/step - loss: 1.6254 - accuracy: 0.2595
Epoch 2/4
72/72 [==============================] - 6s 84ms/step - loss: 1.5326 - accuracy: 0.2991
Epoch 3/4
72/72 [==============================] - 7s 96ms/step - loss: 1.5325 - accuracy: 0.2990
Epoch 4/4
72/72 [==============================] - 7s 91ms/step - loss: 1.5324 - accuracy: 0.2992


In [26]:
y_pred_pro = model.predict(x_train)

In [27]:
y_pred_pro

array([[0.13639387, 0.1736348 , 0.21443917, 0.2631724 , 0.21235974],
       [0.12053148, 0.17371505, 0.22939922, 0.26921672, 0.20713748],
       [0.1170559 , 0.11507253, 0.19598363, 0.3111953 , 0.26069257],
       ...,
       [0.11140109, 0.11954848, 0.19112001, 0.29932374, 0.27860677],
       [0.11140109, 0.11954848, 0.19112001, 0.29932374, 0.27860677],
       [0.11140109, 0.11954848, 0.19112001, 0.29932374, 0.27860677]],
      dtype=float32)

In [28]:
y_pred_class = [np.ceil(np.median(np.where((y_pred_pro[i] >= y_pred_pro[i].mean()))[0])) 
                for i in range(len(y_pred_pro))]

In [29]:
y_pred_orig_model = pd.DataFrame()
y_pred_orig_model['origin'] = np.ravel(y_train)
y_pred_orig_model['predict'] = y_pred_class

### ✅ Check accuracy

In [215]:
y_pred_orig_model['result'] = np.zeros((len(y_pred_orig_model),1))
y_pred_orig_model['result'][abs(y_pred_orig_model['predict'] - y_pred_orig_model['origin']) <= 1] = 1

100 * y_pred_orig_model.result.sum() / len(y_pred_orig_model)

76.82748447007312


In [31]:
import gc
gc.collect()

780

### ✅ Building model (using sklearn library)

In [40]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', hidden_layer_sizes=(20,30,20,30), 
                    activation='tanh', shuffle=True, random_state=1,
                    batch_size=100000,learning_rate='adaptive', verbose=True)

clf.fit(x_train, np.ravel(y_train))

Iteration 1, loss = 1.54007054
Iteration 2, loss = 1.53041011
Iteration 3, loss = 1.52988273
Iteration 4, loss = 1.53015800
Iteration 5, loss = 1.53014494
Iteration 6, loss = 1.53018271
Iteration 7, loss = 1.53039062
Iteration 8, loss = 1.52994977
Iteration 9, loss = 1.52958309
Iteration 10, loss = 1.52978680
Iteration 11, loss = 1.53126947
Iteration 12, loss = 1.53041903
Iteration 13, loss = 1.53174634
Iteration 14, loss = 1.53144766
Iteration 15, loss = 1.53207096
Iteration 16, loss = 1.53144545
Iteration 17, loss = 1.53048696
Iteration 18, loss = 1.53048481
Iteration 19, loss = 1.53047394
Iteration 20, loss = 1.53039251
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(activation='tanh', batch_size=100000,
              hidden_layer_sizes=(20, 30, 20, 30), learning_rate='adaptive',
              random_state=1, verbose=True)

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
    اینبار با استفاده از mlp classifier مدل را تشکیل داده ایم که این مدل نیز شامل 4 لایه نهان با تعداد نورون های 20 و 30 و 20 و 30 می باشد و از activation function tanh و adam solver بهره گیری کرده است.

In [41]:
clf.score(x_train,np.ravel(y_train))

0.2992080363253904

In [44]:
y_pred_pro_clf = clf.predict_proba(x_train)

In [45]:
y_pred_class_clf = [np.ceil(np.median(np.where((y_pred_pro_clf[i] >= y_pred_pro_clf[i].mean()))[0])) 
                    for i in range(len(y_pred_pro_clf))]

In [46]:
y_pred_orig = pd.DataFrame()
y_pred_orig['origin'] = np.ravel(y_train)
y_pred_orig['predict'] = y_pred_class_clf

### ✅ Check accuracy

In [214]:
y_pred_orig['result'] = np.zeros((7148939,1))
y_pred_orig['result'][abs(y_pred_orig['predict'] - y_pred_orig['origin']) <= 1] = 1

100 * y_pred_orig.result.sum() / len(y_pred_orig)

76.50100245644843


In [48]:
import gc
gc.collect()

0

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
همانطور که دیده میشود دقت برابر با 76.5 درصد شده است و همین مدل را وقتی داده ی تست و ترین را جدا کرده بودیم نیز اجرا کردیم و دقت برای داده ی تست نیز 76.8 درصد شده بود اما چون برای حالت کلی باید کل داده ها را اموزش دهیم ، کل داده را به مدل دادیم .

## 📑 Build users  dataset and run model

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
در این قسمت مدل ساخته شده را درصد هستیم که برای یوزرهایی که به عنوان داده ی تست تهیه میشوند تست کنیم .
در قسمت کاوش قواعد توانستیم کتاب ها و نویسنده هایی که باهم مراجعه میشوند
        را بدست آوریم .
        

In [25]:
balanced_final = pd.read_csv(path + 'balanced_classifier_input.csv')

In [26]:
authors_df = pd.read_csv(path + 'author_class.csv')

In [27]:
authors_df['book_id']=[i for i in range(1,10001)]

In [28]:
balanced_result = pd.merge(balanced_final, authors_df, on=['book_id'])

In [50]:
from sklearn.model_selection import train_test_split

x_train = balanced_result.drop(columns=['rating']).copy()
y_train = balanced_result[['rating']].copy()

In [59]:
del balanced_final

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.utils import to_categorical

y_train_1 = to_categorical(y_train)[:,1:].copy()

In [61]:
x_train['u_2'] = x_train.user_id.copy()
x_train['u_3'] = x_train.user_id.copy()
x_train['u_4'] = x_train.user_id.copy()
x_train['u_5'] = x_train.user_id.copy()
x_train['u_6'] = x_train.user_id.copy()
x_train['u_7'] = x_train.user_id.copy()
x_train['u_8'] = x_train.user_id.copy()

In [ ]:
#item base . user base

In [62]:
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=len(x_train.columns), activation='relu',kernel_initializer='random_normal',bias_initializer='zeros'))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(30, activation='relu'))
model.add(Dense(5, activation='softmax'))
# compile the keras model
# myoptimizer = optimizers.Adam(learning_rate=0.001)
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit the keras model on the dataset
history = model.fit(x_train, y_train_1, epochs=4, batch_size=100000,shuffle=True)

Epoch 1/4
72/72 [==============================] - 12s 88ms/step - loss: 1.5986 - accuracy: 0.2871
Epoch 2/4
72/72 [==============================] - 6s 84ms/step - loss: 1.5326 - accuracy: 0.2982
Epoch 3/4
72/72 [==============================] - 6s 84ms/step - loss: 1.5326 - accuracy: 0.2983
Epoch 4/4
72/72 [==============================] - 6s 89ms/step - loss: 1.5326 - accuracy: 0.2983


In [13]:
del x_train
del y_train

In [14]:
author_rules = pd.read_csv(path + 'author_rules.csv').drop(['Unnamed: 0'],axis=1)
book_rules = pd.read_csv(path + 'book_rules.csv').drop(['Unnamed: 0'],axis=1)

In [15]:
import gc
gc.collect()

837

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
یکی از یوزرها را در بازه ی معتبر انتخاب میکنیم و در دیتاست کتاب ، کتاب هایی که خوانده است را در متغیری ریخته و در قواعد بدست آمده بررسی میکنیم که کدام یک
        از مقدم ها زیرمجموعه ی کتبی است که یوزر خوانده است و سپس تالی آن قواعد را در لیستی وارد میکنیم و در واقع مواردی که در این لیست هست را به یوزر پیشنهاد میدهیم .
        در مواردی که هیچ یک از مقدم ها زیرمجموعه کتب خوانده شده نبود، همین الگوریتم را برای تالی اجرا میکنیم .
        در آخر نیز کتاب های پیشنهاد شده را به مدل داده و امتیازی که یوزر احتمال دارد به آنها بدهد را بدست می آوریم و از بین آنها تعدادی را که امتیاز بیشتری گرفته اند به یوزر پیشنهاد میدهیم.

In [134]:
def recommender(read_list, df):  
    recomended_list = set()
    for i in range(len(df)):
        if set(eval(df['antecedents'].loc[i])).issubset(set(read_list)):
            for j in eval(df['consequents'].loc[i]):
                recomended_list.add(j)
    
    if len(recomended_list) == 0:
        for i in range(len(df)):
            if set(eval(df['consequents'].loc[i])).issubset(set(read_list)):
                for j in eval(df['antecedents'].loc[i]):
                    recomended_list.add(j)
    
    return recomended_list.difference(read_list)

In [218]:
authors_read = [ 'David McDuff', 'Zadie Smith', 'Zadie Smith', 'Amy Tan', 'Amy Tan', 'Amy Tan', 'Amy Tan', 
                'Margaret Atwood', 'Margaret Atwood', 'Jane Austen', 'Tony Tanner', 'Ros Ballaster', 
                'Virginia Woolf', 'Jonathan Safran Foer', 'Jane Austen', 'Kathryn Sutherland', 'John Seelye']

recomended_list = recommender(authors_read, author_rules)
print(recomended_list)
print('______________________________________')
print(len(recomended_list), ' recomended item')

{'Mark Twain', 'J.K. Rowling', 'Guy Cardwell'}
______________________________________
3  recomended item


In [213]:
b = ['8452', '522', '4788', '150', '9962', '9790', '1283', '9049', '9215', '4', 
    '6745', '247', '1378', '420', '815', '104', '6212', '1142', '184', '8389', '1025', '4715', '8492', '977', 
    '2615', '265', '2190', '2403', '905', '2963', '9966', '3700', '773', '715', '782', '599', '4290', '660', 
    '9533', '1953', '1598', '501', '427', '7401', '138', '229', '77', '738', '6607', '4686', '5347', '410', '117', 
    '59', '484', '177', '1928', '1005', '118', '1714', '4057', '3311', '5395', '883', '76', '1664', '587', '471', 
    '3229', '2235', '101', '28', '765', '55', '119', '5932', '2010', '13', '1078', '1929', '9014', '1068', '2680', 
    '531', '301', '659', '4090', '32', '58', '15', '255', '5448', '3566', '2439', '932', '1146', '726', '8403', '4582', 
    '6274', '5765', '10', '233', '66']

recomended_list = recommender(b, book_rules)
print(recomended_list)
print('______________________________________')
print(len(recomended_list), ' recomended item')

{'35', '7', '1', '65', '63', '17', '23', '27', '54', '33', '57', '18', '21', '47', '19', '5', '45', '43', '22', '46', '29', '30', '3', '20', '60', '48', '38', '37', '8', '36', '31', '2', '11', '25', '14', '9', '16', '24', '6', '26', '42'}
______________________________________
41  recomended item


In [203]:
    
import re
def recom_system(u_id):
    authors_read = []
    
    x = rating[rating['user_id'] == u_id]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        authors = re.split(',', book.loc[_bid_ -1, 'authors'])
        for _au_ in authors:
            authors_read.append(_au_.lstrip())
        
    x = to_read[to_read['user_id'] == u_id]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        authors = re.split(',', book.loc[_bid_ -1, 'authors'])
        for _au_ in authors:
            authors_read.append(_au_.lstrip())
            
    #print('authors_read', authors_read)
    author_recomended_list = recommender(authors_read, author_rules)
    #print(author_recomended_list)
    # -------------------------------------------------------------- #

    books_read = []
    x = rating[rating['user_id'] == u_id]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        books_read.append(str(_bid_))
    
    x = to_read[to_read['user_id'] == u_id]
    reads = x['book_id'].unique()
    for _bid_ in reads:
        books_read.append(str(_bid_))
    # -------------------------------------------------------------- #

    book_recom_author = []
    for i in range(len(book)):
        auth_list = re.split(',',book.loc[i,'authors'])
        if set(auth_list).issubset((author_recomended_list)):
            book_recom_author.append(book.loc[i,'book_id'])
    # -------------------------------------------------------------- #
    #print('book_read', books_read)
    book_recomended_list = recommender(books_read, book_rules)
    #print('book_recomended_list', book_recomended_list)
    # -------------------------------------------------------------- #
    
    final_recom_book = set()
    final_recom_book.update(set(book_recomended_list))
    final_recom_book.update(set(book_recom_author))
    #final_recom_book = [int(i)-1 for i in final_recom_book]
    #print('\n\n')
    #print(final_recom_book)
    #print(len(final_recom_book))
    # -------------------------------------------------------------- #
    
    test_user_1 = pd.DataFrame(columns=balanced_result.columns)
    
    for _bid_ in final_recom_book:
        selected = balanced_result[balanced_result['book_id'] == int(_bid_)]
        test_user_1 = test_user_1.append(selected.iloc[0]) 

    test_user_1['user_id'] = [u_id for j in range(len(test_user_1))]
    
    test_user_1 = test_user_1.drop_duplicates(subset=['book_id'], keep='first')
    #display(test_user_1)
    
     # -------------------------------------------------------------- #
    
    test_user_1['u_2'] = test_user_1.user_id.copy()
    test_user_1['u_3'] = test_user_1.user_id.copy()
    test_user_1['u_4'] = test_user_1.user_id.copy()
    test_user_1['u_5'] = test_user_1.user_id.copy()
    test_user_1['u_6'] = test_user_1.user_id.copy()
    test_user_1['u_7'] = test_user_1.user_id.copy()
    test_user_1['u_8'] = test_user_1.user_id.copy()
    
    test_user_1.drop(columns=['rating'], inplace=True)
    #display((test_user_1).sort_index())    
    if len(test_user_1) == 0:
        return
    recom_user_1 = model.predict(test_user_1)
    y_pred_class_model = [np.ceil(np.median(np.where((recom_user_1[i] >= recom_user_1[i].mean()))[0])) 
                          for i in range(len(recom_user_1))]
    
    data = pd.DataFrame(y_pred_class_model)

    print('---------------------------------------------------------')
    final_return = pd.DataFrame(columns=['book_id','rate'])
    final_return.book_id = test_user_1.book_id
    final_return = pd.merge(final_return,book[['book_id','title']], on=['book_id'])

    final_return.rate = np.array(y_pred_class_model) + 1
    final_return = final_return.sort_values(by=['rate'],ascending=False)
    final_return.drop_duplicates(inplace=True)
    return final_return.iloc[:10]

#     recom_user_1 = model.predict(test_user_1)
#     recom_user_clss = model.predict_classes(test_user_1)
#     print(recom_user_clss)
#     print('________________________',u_id,'________________________')

#     data = pd.DataFrame(recom_user_clss)
#     print(data.value_counts())
#     print('---------------------------------------------------------')
#     print(" class in model :")
#     print(recom_user_clss)
#     print('---------------------------------------------------------')
#     print(" probability in model :")
#     print(recom_user_1)

## 💎 Real Test & Deployment Phase

<hr style="border:2px solid blue"> </hr>

<div style="direction:rtl", align="justify"><h3>
    <span style='font-family:"B Nazanin"'>
        در این قسمت 10 تا از یوزرها را از یک تا 10 انتخاب کرده و الگوریتم را برایش اجرا میکنیم .
        به ازای هر یوزر کتابهایی که پیشنهاد شده و نیز امتیاز پیش بینی شده ای که احتمال دارد آن یوزر به کتاب مربوطه بدهد را نمایش میدهیم . 

In [204]:
for i in range(1, 10):
    print('recomended books for user with id =', i)
    display(recom_system(i))

recomended books for user with id = 1
---------------------------------------------------------


,book_id,rate,title
0,7.0,4.0,The Hobbit
16,2.0,4.0,Harry Potter and the Sorcerer's Stone (Harry P...
35,29.0,4.0,Romeo and Juliet
34,26.0,4.0,"The Da Vinci Code (Robert Langdon, #2)"
33,6.0,4.0,The Fault in Our Stars
32,55.0,4.0,Brave New World
31,5.0,4.0,The Great Gatsby
30,24.0,4.0,Harry Potter and the Goblet of Fire (Harry Pot...
28,3.0,4.0,"Twilight (Twilight, #1)"
27,19.0,4.0,The Fellowship of the Ring (The Lord of the Ri...


recomended books for user with id = 2
---------------------------------------------------------


,book_id,rate,title
0,1.0,4.0,"The Hunger Games (The Hunger Games, #1)"
26,41.0,4.0,The Lightning Thief (Percy Jackson and the Oly...
39,6.0,4.0,The Fault in Our Stars
41,32.0,4.0,Of Mice and Men
44,35.0,4.0,The Alchemist
58,22.0,4.0,The Lovely Bones
59,55.0,4.0,Brave New World
68,36.0,4.0,"The Giver (The Giver, #1)"
1,3.0,4.0,"Twilight (Twilight, #1)"
78,63.0,4.0,Wuthering Heights


recomended books for user with id = 3
---------------------------------------------------------


...

,book_id,rate,title
29,56.0,5.0,"Breaking Dawn (Twilight, #4)"
13,20.0,5.0,"Mockingjay (The Hunger Games, #3)"
44,2.0,5.0,Harry Potter and the Sorcerer's Stone (Harry P...
31,45.0,5.0,Life of Pi
30,22.0,5.0,The Lovely Bones
1,1.0,5.0,"The Hunger Games (The Hunger Games, #1)"
28,49.0,5.0,"New Moon (Twilight, #2)"
25,47.0,5.0,The Book Thief
48,15.0,5.0,The Diary of a Young Girl
23,52.0,5.0,"Eclipse (Twilight, #3)"


recomended books for user with id = 8
---------------------------------------------------------


,book_id,rate,title
38,15.0,5.0,The Diary of a Young Girl
22,20.0,5.0,"Mockingjay (The Hunger Games, #3)"
20,3.0,5.0,"Twilight (Twilight, #1)"
1,7.0,5.0,The Hobbit
37,9.0,5.0,"Angels & Demons (Robert Langdon, #1)"
17,46.0,5.0,Water for Elephants
16,22.0,5.0,The Lovely Bones
15,47.0,5.0,The Book Thief
29,8.0,5.0,The Catcher in the Rye
13,21.0,5.0,Harry Potter and the Order of the Phoenix (Har...


recomended books for user with id = 9
---------------------------------------------------------


,book_id,rate,title
0,4.0,5.0,To Kill a Mockingbird
14,57.0,5.0,The Secret Life of Bees
106,61.0,5.0,The Girl on the Train
86,69.0,5.0,"Insurgent (Divergent, #2)"
72,53.0,5.0,"Eragon (The Inheritance Cycle, #1)"
62,54.0,5.0,The Hitchhiker's Guide to the Galaxy (Hitchhik...
1,12.0,5.0,"Divergent (Divergent, #1)"
53,62.0,5.0,"The Golden Compass (His Dark Materials, #1)"
49,6.0,5.0,The Fault in Our Stars
47,67.0,5.0,A Thousand Splendid Suns
